# Reproducing in-text result 3

This result starts with the doctor utilisation from Figure 2A which is over 100% when patient load is 170 (IAT 3) and appointment times are 5 minutes (mean 5, SD 1, boundary 2).

The variant introduced is to reduce the doctor intervention during delivery so that:

* 50% delivery cases: no doctor
* 30% delivery cases: doctor but only one-third of typical intervention
* 20% delivery cases: doctor and full intervention

## Set up

In [21]:
# To run model
import PHC
from reproduction_helpers import process_results

# To import results and produce figures
import xlrd
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

# To speed up run time
from multiprocessing import Pool

In [22]:
# Paths to save image files to
output_folder = 'outputs'
save_path = os.path.join(output_folder, 'intext3.csv')

## Run model

In [23]:
# Parameters used in both models
base_model = {
    'OPD_iat': 3,
    'rep_file': 'arr170',
    'mean': 5,
    'sd': 1,
    'consult_boundary_1': 2,
    'consult_boundary_2': 2
}

# Model variants
variants = [
    {
        'doctor_delivery_scenario': False,
        'rep_file': 'in3_normal_delivery.xls'
    },
    {
        'doctor_delivery_scenario': True,
        'rep_file': 'in3_reduced_delivery.xls'
    },
]

In [24]:
# Combine dictionaries
dict_list = []
for var in variants:
    dict_list.append({**base_model, **var})

# Append 's_' to all items
for i, d in enumerate(dict_list):
    dict_list[i] = {f's_{k}': v for k, v in d.items()}

# View dictionaries
dict_list

[{'s_OPD_iat': 3,
  's_rep_file': 'in3_normal_delivery.xls',
  's_mean': 5,
  's_sd': 1,
  's_consult_boundary_1': 2,
  's_consult_boundary_2': 2,
  's_doctor_delivery_scenario': False},
 {'s_OPD_iat': 3,
  's_rep_file': 'in3_reduced_delivery.xls',
  's_mean': 5,
  's_sd': 1,
  's_consult_boundary_1': 2,
  's_consult_boundary_2': 2,
  's_doctor_delivery_scenario': True}]

In [25]:
'''# Wrapper function to allow input of dictionary with pool
def wrapper(d):
    return PHC.main(**d)

# Create a process pool that uses all CPUs
with Pool() as pool:
    # Run PHC.main() using each of inputs from config
    pool.map(wrapper, dict_list)'''

'# Wrapper function to allow input of dictionary with pool\ndef wrapper(d):\n    return PHC.main(**d)\n\n# Create a process pool that uses all CPUs\nwith Pool() as pool:\n    # Run PHC.main() using each of inputs from config\n    pool.map(wrapper, dict_list)'

## Process results

In [26]:
data = process_results([i['s_rep_file'] for i in dict_list], xls=True)

In [29]:
def get_utilisation(index_name, new_name):
    '''
    Creates dataframe with columns for utilisation (for a given row) before and
    after scenario change.

    Parameters:
    -----------
    index_name : string
        Name of row with utilisation data
    new_name : string
        To rename the utilisation row (e.g. doc occ -> doctor utilisation)

    Returns:
    --------
    util : pandas DataFrame
        Dataframe with three columns for utilisation before and after admin,
        and change in utilisation.
    '''
    # Get utilisation
    util = round(pd.DataFrame(data.loc[index_name]).T, 3)

    # Find change
    util['Change'] = util['in3_reduced_delivery'] - util['in3_normal_delivery']

    # Rename index
    util = util.rename_axis('Output')
    # Rename row
    util = util.rename({index_name: new_name})
    # Rename columns for clarity for readers
    util = util.rename({
        'in3_normal_delivery': 'Normal delivery',
        'in3_reduced_delivery': 'Reduced doctor intervention'}, axis=1)

    return util

In [30]:
# Get results for doctor utilisation and staff nurse utilisation
util = pd.concat([
    get_utilisation('doc occ', 'Doctor utilisation'),
    get_utilisation('staff nurse occ', 'Staff nurse utilisation')])

# Display results
display(util)

# Save results
util.to_csv(save_path, index=True)

,Normal delivery,Reduced doctor intervention,Change
Output,,,
Doctor utilisation,1.142,1.131,-0.011
Staff nurse utilisation,0.323,0.321,-0.002
